In [1]:
%load_ext autoreload
%autoreload 2
from TNR import *

In [120]:
# htemp=KP(s0,sx,sx,s0)+KP(s0,sy,sy,s0)+KP(s0,sz,sz,s0)
# htemp=KP(sx,sx,s0,s0)+KP(sy,sy,s0,s0)+KP(sz,sz,s0,s0)
# htemp=KP(sz,sz)+(KP(sx,s0)+KP(s0,sx))/4

# d=int(np.sqrt(htemp.shape[0]))
# Ainit=expm(-htemp).reshape(d,d,d,d)
# A0=Asplit(Ainit,chi)

na=np.newaxis
s=np.array([1,-1])
Tc=2/np.log(1+np.sqrt(2))
T=0.995*Tc
A0=np.exp(-1/T*(s[:,na,na,na]*s[na,:,na,na]+s[na,:,na,na]*s[na,na,:,na]\
                +s[na,na,:,na]*s[na,na,na,:]+s[na,na,na,:]*s[:,na,na,na]))
A0/=np.linalg.norm(A0)

chi={}
chi['w'],chi['y'],chi['u'],chi['v']=6,6,6,6
chiw,chiy,chiu,chiv=chi['w'],chi['y'],chi['u'],chi['v']

In [121]:
RGsteps=15
A_list=[None]*(RGsteps+1)
Anorm_list=[None]*(RGsteps+1)
B_list,U_list,vl_list,vr_list,yl_list,yr_list,D_list,w_list,u_list,v_list=[[None]*RGsteps for i in range(10)]
A_list[0]=A0

for i in range(RGsteps):
    print('RG step: ',i)
    Aold=A_list[i]
    B,U,vl,vr,yl,yr,D,w,A,Anorm=doTNR(Aold,chi)
    if A.shape==Aold.shape:
        A,u,v=fix_gauge(Aold,A,chi)
        u_list[i],v_list[i]=u,v
    B_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],A_list[i+1],Anorm_list[i+1]=\
    B,U,vl,vr,yl,yr,D,w,A,Anorm
    print()

RG step:  0
iter=0, U.Uenv=0.578607
iter=100, U.Uenv=0.578608
iter=200, U.Uenv=0.578608
iter=300, U.Uenv=0.578608
iter=400, U.Uenv=0.578608
iter=500, U.Uenv=0.578608
iter=600, U.Uenv=0.578608
iter=700, U.Uenv=0.578608
iter=800, U.Uenv=0.578608
iter=900, U.Uenv=0.578608
iter=1000, U.Uenv=0.578608
error in U, vl, vr optimization: -1.71044e-16
error in yl, yr, D optimization: 0.00242952
error in w optimization: 0.0423569

RG step:  1
iter=0, U.Uenv=0.487936
iter=100, U.Uenv=0.488354
iter=200, U.Uenv=0.488355
iter=300, U.Uenv=0.488355
iter=400, U.Uenv=0.488355
iter=500, U.Uenv=0.488355
iter=600, U.Uenv=0.488355
iter=700, U.Uenv=0.488355
iter=800, U.Uenv=0.488355
iter=900, U.Uenv=0.488355
iter=1000, U.Uenv=0.488355
error in U, vl, vr optimization: 0.00527209
error in yl, yr, D optimization: 0.013139
error in w optimization: 0.0220124
iter=0, u.uenv=0.330044, v.venv=0.921307
iter=1000, u.uenv=0.970341, v.venv=0.970341
iter=2000, u.uenv=0.970341, v.venv=0.970341
Adiff 0.243553

RG step:  2
it

In [4]:
# def Az_(Ainit,chi):
#     #input: Ainit, chi, op: operator acting on a leg
#     #output:up-down symmetric Aout
#     chiw,chiy,chiu,chiv=chi['w'],chi['y'],chi['u'],chi['v']
#     As=Ainit.shape
    
#     #vertical cut
#     Av=Ainit.reshape(As[0]*As[1],As[2]*As[3])
#     dtemp,vtemp=eigCut(Av,chiw)
#     v=(vtemp*np.sqrt(dtemp)).reshape(As[0],As[1],min(chiw,As[0]*As[1]))

#     #horizontal cut
#     vl2,w,vr2= h_cut(Ainit,chiy)
#     Aout=np.einsum(vr2,[4,5,0],v,[5,7,1],vl2*w,[6,7,2],v.conj(),[4,6,3])
#     Azout=0.5*(np.einsum(vr2,[4,5,0],v,[5,7,1],sz@vl2*w,[6,7,2],v.conj(),[4,6,3])+np.einsum(vr2,[4,5,0],v,[5,7,1],vl2*w,[6,7,2],sz@v.conj(),[4,6,3]))
#     Anorm=np.linalg.norm(Aout)
#     return Azout/Anorm

# A0=Asplit(Ainit,chi)
# Az=Az_(Ainit,chi)
# def MZ_(A0,Az):
#     return np.einsum(Az,[2,9,8,0],A0,[8,10,3,1],A0,[4,6,11,9],A0,[11,7,5,10])

In [122]:
def MA_(A0):
    return np.einsum(A0,[2,9,8,0],A0,[8,10,3,1],A0,[4,6,11,9],A0,[11,7,5,10])
def MZ_(A0):
    return np.einsum(A0,[2,9,8,0],A0,[8,10,3,1],A0,[4,6,11,9],A0*s,[11,7,5,10])
def Mtrace_(MA):
    return np.einsum(MA,[0,1,2,2,3,3,0,1])
MA_list=[None]*(RGsteps+1)
MZ_list=[None]*(RGsteps+1)
MA=MA_(A0)
MAtrace=Mtrace_(MA)
MA_list[0]=MA/MAtrace
MZ_list[0]=MZ_(A0)/MAtrace

In [140]:
for i in range(RGsteps):
    MAold=MA_list[i]
    MZold=MZ_list[i]
    A,U,vl,vr,yl,yr,D,w,u,v=\
    A_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],u_list[i],v_list[i]
    MA=Mscaled_(MAold,A,U,vl,vr,yl,yr,D,w,u,v)
    MZ=Mscaled_(MZold,A,U,vl,vr,yl,yr,D,w,u,v)
    MAtrace=Mtrace_(MA)
    MA_list[i+1]=MA/MAtrace
    MZ_list[i+1]=MZ/MAtrace
    print('mag=',chop(Mtrace_(MZ/MAtrace)))
    #print('1=',Mtrace_(MA/MAtrace))

mag= (2.8985216636029737e-06+0j)
mag= (-0.0008387623138663736+0j)
mag= (-0.004233037568655102+0j)
mag= (-0.016615117373104164+0j)
mag= (-0.06241330508918931+0j)
mag= (-0.2299553212083796+0j)
mag= (-0.5723512947821574+0j)
mag= (-0.6307746898709389+0j)
mag= (-0.6307818510373334+0j)
mag= (-0.6307818510373334+0j)
mag= (-0.6307818510373315+0j)
mag= (-0.6307818510373326+0j)
mag= (-0.630781851037332+0j)
mag= (-0.6307818510373323+0j)
mag= (-0.6307818510373345+0j)


In [113]:
#exact value
mag_exact=(1-np.sinh(2/T)**(-4))**(1/8)
print(mag_exact)

0.629648162973007


scaling dimensions

In [132]:
i=3
A=A_list[i+1]
Aold,U,vl,vr,yl,yr,D,w,u,v=A_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],u_list[i],v_list[i]
def wrenv_(A):
    return np.einsum(A,[5,6,0,8],A,[4,7,1,6],A.conj(),[4,7,3,9],A.conj(),[5,9,2,8])
def wlenv_(A):
    return np.einsum(A,[0,6,5,8],A,[1,7,4,6],A.conj(),[3,7,4,9],A.conj(),[2,9,5,8])
def wvenv_(A):
    return np.einsum(A,[8,0,6,5],A,[6,1,7,4],A.conj(),[8,2,9,5],A.conj(),[9,3,7,4])
chik=20
wrenv=wrenv_(A)
_,wr=compress(wrenv,chik)
wlenv=wlenv_(A)
_,wl=compress(wlenv,chik)
wvenv=wvenv_(A)
_,wv=compress(wvenv,chik)

wrenv=wrenv_(Aold)
_,wrold=compress(wrenv,chik)
wlenv=wlenv_(Aold)
_,wlold=compress(wlenv,chik)
wvenv=wvenv_(Aold)
_,wvold=compress(wvenv,chik)

In [134]:
def gauge_(gnw,gne,u,v):
    if type(u)==type(None):
        return gnw,gne
    else:
        gnw_gauged=np.einsum(gnw@u,[0,1,4,3],v.conj(),[4,2])
        gne_gauged=np.einsum(gne@u,[0,1,4,3],v,[4,2])
        return gnw_gauged,gne_gauged

In [135]:
gl=gl_(vl)
gr=gr_(vr)
gu=gu_(Aold,U,vl,vr)
gnw=gnw_(vl,vr,yr,D,w)
gne=gne_(vl,vr,yl,D,w)
gnw,gne=gauge_(gnw,gne,u,v)

In [136]:
glw=np.einsum(gl,[3,4,1,2],wrold.conj(),[3,4,0])
grw=np.einsum(gr,[3,4,1,2],wlold.conj(),[3,4,0])
guw=np.einsum(gu,[3,4,1,2],wvold.conj(),[3,4,0])
def scaleop(v):
    v4=v.reshape(chik,chik,chik,chik)
    out=np.einsum(v4,[8,9,10,11],glw,[10,19,21],grw,[8,18,20],guw,[9,22,23],guw.conj(),[11,16,17],gnw,[20,22,5,14],gne,[21,23,7,15],gnw.conj(),[18,16,4,12],gne.conj(),[19,17,6,13],wl,[4,5,0],wr,[6,7,2],wv,[14,15,1],wv.conj(),[12,13,3],optimize=('greedy',2**100))
    return out.flatten()
def rscaleop(v):
    v4=v.reshape(chik,chik,chik,chik)
    out=np.einsum(v4.conj(),[0,1,2,3],glw,[10,19,21],grw,[8,18,20],guw,[9,22,23],guw.conj(),[11,16,17],gnw,[20,22,5,14],gne,[21,23,7,15],gnw.conj(),[18,16,4,12],gne.conj(),[19,17,6,13],wl,[4,5,0],wr,[6,7,2],wv,[14,15,1],wv.conj(),[12,13,3],optimize=('greedy',2**100))
    return out.flatten().conj()

In [137]:
import scipy
from scipy.sparse.linalg import LinearOperator

n=chik**4
N_level=100
Atemp = LinearOperator((n,n), matvec = scaleop, rmatvec = rscaleop, dtype='complex')

In [131]:
ew = scipy.sparse.linalg.svds(Atemp, k=N_level, which='LM', maxiter=200, tol=1e-5, return_singular_vectors=False)
ew=ew[::-1]
-np.log2(np.abs(ew/ew[0]))

array([-0.        ,  0.09603869,  0.91314976,  1.03848894,  1.06049471,
        1.88906926,  1.8310601 ,  1.78338902,  1.79179025,  1.8029546 ,
        1.80086261,  2.05807152,  2.6474452 ,  2.67695708,  2.68392507,
        2.69090698,  2.69820594,  2.71221363,  2.71553667,  2.7839871 ,
        2.86627828,  2.88596264,  2.90992285,  3.47578695,  3.86478737,
        3.52927954,  3.53658895,  3.72750857,  3.54894076,  3.55323468,
        3.5667369 ,  3.68394283,  3.68572378,  3.63953118,  3.63170705,
        3.61009888,  3.59948529,  3.5964812 ,  3.59310579,  3.59370726,
        4.06587343,  4.3095935 ,  4.33300921,  4.39531287,  4.41209089,
        4.4227364 ,  4.44155912,  4.44520043,  4.48451282,  4.48828176,
        4.51632999,  4.51225545,  4.50742162,  4.5083088 ,  4.53813752,
        4.58233647,  4.58756364,  4.59119291,  4.62191793,  4.63078248,
        4.63639507,  4.64094846,  4.67124122,  4.69192808,  4.71518756,
        4.87547209,  4.92198759,  5.11087829,  5.14787624,  5.29

In [139]:
ew = scipy.sparse.linalg.eigs(Atemp, k=N_level, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew=ew[::-1]
-np.log2(np.abs(ew/ew[0]))